In [ ]:
import pandas as pd
import zipfile
import os
import time
from google.colab import drive
from PIL import Image

In [ ]:
!pip install pysrt
import pysrt

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [224]:
# чтобы разгрузить код я буду путь к файлам сохранять в эту переменную
PATH = None

In [225]:
# создаем датафрейм
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/movies_labels.xlsx'
data = pd.read_excel(PATH)
data.head()

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [226]:
data.shape[0]

241

In [227]:
# читаем файл с субтитрами
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/Subtitles/10_Cloverfield_lane(2016).srt'
subs = pysrt.open(PATH)


In [228]:
#help (pysrt)

In [229]:
# я посмотрела как будут выглядеть субтитры
subtitle_texts = [subtitle.text for subtitle in subs]
#subtitle_texts

Для того, чтобы получить субтитры в удобноваримом формате, мы пройдемся циклом по всему содержимому папки Subtitles, которая содержит папки с названиями фильмов. Для этого используем модуль os, который позволяет работать с файлами и директориями.

Я выбирала между сохранить в словарь название:субтитры или два списка.
Решила сделать два списка, потому что потом мне нужно будет сравнивать фильмы в дф и в списке фильмов с субтитрами, а это легче сделать из списка, а потом добавить в дф.

In [230]:
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/'

In [233]:
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/'

def get_subs():
    subtitles = []
    movie = []

    # Получаем все папки и файлы в указанной папке
    film_list = os.listdir(PATH)

    for folder in film_list:
        for film in os.listdir(os.path.join(PATH, folder)):
            # достаем названия
            last_dot = film.rfind('.')
            movie.append(film[:last_dot])

            # субтитры
            # собираем название путь
            file_to_read = os.path.join(PATH, folder, film)  # Исправленный путь к файлу
            # расшифровываем
            try:
                encoded_film = pysrt.open(file_to_read, encoding='iso-8859-1')
            except:
                encoded_film = pysrt.open(file_to_read, encoding='UTF-8')

            # трансформируем в текст
            subs_text = [film.text for film in encoded_film]
            subtitles.append(subs_text)

    return pd.DataFrame({'subs': subtitles, 'Movie': movie})


Обращаем внимание, что имеются строки в которых точка используется как разделитель, это важно, так как нам нужно сохранить названия фильмов без расширения .srt

---

Делить будем так:
- находим индекс где стоит **последняя** точка
- добавляем строку до этого индекса

В работе цикла возникла ошибка что файл не найден в указанной директории.
Я попробовала открыть отдельно этот файл(вручную прописав путь к нему) и возникла ошибка кодировки. Чтобы решить эти вопросы:
- Поэтому добавила в цикл try-except, чтобы обработать все возможные кодировки
- Создала переменную путь и название, чтобы избежать того, что возникает ошибка из-за некорректной обработки названий.


In [216]:
subs_df = get_subs()
subs_df.head()

FileNotFoundError: ignored

In [234]:
subs_df.shape[0]

278

Теперь можно обработать названия фильмов, чтобы добавить субтитры к фильмам.
Обрабатывать буду следующими шагами:
- все слова к строчному регистру
- в качестве разделителя только знак нижнего подчеркивания "_"
- удалю все пробелы, знаки между словами, оставлю только цифры.

In [236]:
data.head()

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [235]:
def transform_values(df):
    # заменяем пробелы на underscore
    df = df.applymap(lambda x: x.replace(' ', '_') if isinstance(x, str) else x)

    # заменяем дефис на underscore
    df = df.applymap(lambda x: x.replace('-', '_') if isinstance(x, str) else x)

    # добавляем underscore между словом и цислом
    df = df.applymap(lambda x: '_'.join(part for part in x.split() if not part.isdigit()) if isinstance(x, str) else x)
    # приводим к строчному регистру
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    return df


In [237]:
# почистим названия фильмов в дата
data = transform_values(data)
data.head()

,id,Movie,Level
0,0,10_cloverfield_lane(2016),b1
1,1,10_things_i_hate_about_you(1999),b1
2,2,a_knights_tale(2001),b2
3,3,a_star_is_born(2018),b2
4,4,aladdin(1992),a2/a2+


In [153]:
data.columns

Index(['id', 'Movie', 'Level'], dtype='object')

In [238]:
subs_df = transform_values(subs_df)
subs_df.head()

,subs,Movie
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english


In [239]:
subs_df['Movie'] = transform_values(subs_df[['Movie']])
subs_df['Movie'].head()

0             the_walking_dead_s01e05_wildfire.english
1                the_walking_dead_s01e06_ts_19.english
2                the_walking_dead_s01e04_vatos.english
3    the_walking_dead_s01e03_tell_it_to_the_frogs.e...
4                 the_walking_dead_s01e02_guts.english
Name: Movie, dtype: object

In [240]:
subs_df.head()

,subs,Movie
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english


Сделаю тип объединения outer так как могут удалиться дубликаты, а я хочу сохранить все наблюдения и отдельно обработать дубликаты чтобы избежать ошибок.

In [242]:
data = pd.merge (subs_df,data,
                 on = 'Movie',
                 how = 'outer',
                 indicator = True)
data.head()

,subs,Movie,id,Level,_merge
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english,116.0,a2,both
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english,117.0,a2,both
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english,115.0,a2,both
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...,114.0,a2,both
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english,113.0,a2,both


In [243]:
data['_merge'].value_counts()

both          233
left_only      49
right_only      8
Name: _merge, dtype: int64

видим что у нас остались 49 фильмов только из датафрейма subs_df и 8 только из data.
очевидно, что для 49 фильмов у нас нет разметки по уровню и для 8 нет субтитров.

In [247]:
data.loc[(data['_merge']=='left_only') | (data['_merge']=='right_only')]

,subs,Movie,id,Level,_merge
38,"[On the clothes we wear,, the people who make ...",the.true.cost.2015.bluray.720p.700mb.ganool.com,NaN,NaN,left_only
39,"[Yes., Ah, there you are. Come in., I'm practi...","crown,_the_s01e05___smoke_and_mirrors.en",NaN,NaN,left_only
40,"[[woman speaks French] Your Majesty., [planes ...","crown,_the_s01e07___scientia_potentia_est.en.sdh",NaN,NaN,left_only
41,[[music plays]\n<i>...and then I swing you aro...,"crown,_the_s01e10___gloriana.en.sdh",NaN,NaN,left_only
42,"[[child]<i> Gina! Gina! Ella!</i>, -Gina! Cari...","crown,_the_s01e03___windsor.en.sdh",NaN,NaN,left_only
43,"[- Fuel on.\n- Fuel on., Chocks are in positio...","crown,_the_s01e04___act_of_god.en",NaN,NaN,left_only
44,"[[fanfare plays], [man] <i>More than half a mi...","crown,_the_s01e06___gelignite.en.sdh",NaN,NaN,left_only
45,"[[coughing], [coughing continues], [spits], [e...","crown,_the_s01e01___wolferton_splash.en.sdh",NaN,NaN,left_only
46,"[<i>...and then I swing you around there.</i>,...","crown,_the_s01e10___gloriana.en",NaN,NaN,left_only
47,"[<i>âª If your sweetheart âª</i>, <i>âª Sen...","crown,_the_s01e09___assassins.en.sdh",NaN,NaN,left_only


In [ ]:
!git init

In [ ]:
!git clone https://github.com/AV-BOLT/Masterskaya_2.git

fatal: destination path 'Masterskaya_2' already exists and is not an empty directory.


In [ ]:
!git remote add origin https://github.com/AV-BOLT/Masterskaya_2.git

In [ ]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	English_scores.zip
	English_scores.zip (Unzipped Files)/
	Masterskaya_2/
	Notebooks/

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
ls


 English_scores.zip                      Masterskaya_2/
'English_scores.zip (Unzipped Files)'/   Notebooks/


In [ ]:
!git commit -m 'Создала файл ноутбука'

[master 8fada91] Создала файл ноутбука
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:
!git config --global user.email "gelika176@.com"
!git config --global user.name "Anzhelika Boltneva"

In [ ]:
!git add .

In [ ]:
!git commit -m 'Создала новый репозиторий и перезалила ноутбук'

[master (root-commit) 3c7ba11] Создала новый репозиторий и перезалила ноутбук
 1 file changed, 1 insertion(+)
 create mode 100644 determine_langlevel_by_subs.ipynb


In [ ]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   determine_langlevel_by_subs.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add determine_langlevel_by_subs.ipynb

In [ ]:
!git status

:wq
On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   determine_langlevel_by_subs.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   determine_langlevel_by_subs.ipynb



In [ ]:
!pwd

'/content/drive/MyDrive/Masterskaya_2/Notebooks'

# Новый раздел